In [371]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [388]:
import csv
from collections import defaultdict
import json 
import pandas as pd
import pickle
from play_scraper import api
from play_scraper import googleplaystore

In [375]:
api.collection('TOP_FREE', 'HEALTH_AND_FITNESS', results=10)

new


[{'app_id': 'com.deepsleep.sleep.soft.music.sounds',
  'description': u'\u23f0Mediate and relax with the relaxing sounds to find inner peace and calmness.',
  'developer': 'Sound Sleep Group',
  'developer_id': 'Sound+Sleep+Group',
  'free': False,
  'full_price': None,
  'icon': 'https://lh3.googleusercontent.com/6PVjbcAHiYCUHzgiRYg2Ti3TOk1URJ5EIDE6JWi6OBdCsJ2Xi7J9GB6IriHm2nEQjuug',
  'price': u'',
  'score': '4.3',
  'title': 'Relax Music - Meditation& Sleep Music, White Noise',
  'url': 'https://play.google.com/store/apps/details?id=com.deepsleep.sleep.soft.music.sounds'},
 {'app_id': 'step.tracker.stepcounter.walking',
  'description': u'Free step counter, Daily Walking tracker.  Effective pedometer app in steps\U0001f525',
  'developer': 'Sound Sleep Group',
  'developer_id': 'Sound+Sleep+Group',
  'free': False,
  'full_price': None,
  'icon': 'https://lh3.googleusercontent.com/1aSOiuhycGX5lUVlogYJshrwynHwDr9wDpB-2vXB_RtYvQBfuagXhE8ABel-J056mQ',
  'price': u'',
  'score': '4.6',


In [381]:
top_paid =  [item['app_id'] for item in api.collection('TOP_PAID', 'HEALTH_AND_FITNESS', results=120)]
top_free =  [item['app_id'] for item in api.collection('TOP_FREE', 'HEALTH_AND_FITNESS', results=120)]
new_paid =  [item['app_id'] for item in api.collection('NEW_PAID', 'HEALTH_AND_FITNESS', results=120)]
new_free =  [item['app_id'] for item in api.collection('NEW_FREE', 'HEALTH_AND_FITNESS', results=120)]
top_grossing =  [item['app_id'] for item in api.collection('TOP_GROSSING', 'HEALTH_AND_FITNESS', results=120)]
trending =  [item['app_id'] for item in api.collection('TRENDING', 'HEALTH_AND_FITNESS', results=120)]

new
new
new
new
new
new


In [396]:
with open('top_paid.outfile', 'wb') as fp:
    pickle.dump(top_paid, fp)
with open('top_free.outfile', 'wb') as fp:
    pickle.dump(top_free, fp)
with open('new_paid.outfile', 'wb') as fp:
    pickle.dump(new_paid, fp)
with open('new_free.outfile', 'wb') as fp:
    pickle.dump(new_free, fp)
with open('top_grossing.outfile', 'wb') as fp:
    pickle.dump(top_grossing, fp)
with open('trending.outfile', 'wb') as fp:
    pickle.dump(trending, fp)

In [390]:
with open ('top_paid.outfile', 'rb') as fp:
    itemlist = pickle.load(fp)

In [397]:
def get_reviews(app_id, num_pages=10):
    ps = googleplaystore.PlayStore(cookie={"PLAY_PREFS":"",
          "NID":"",
          "_gat":"",
          "_ga":""
    })
    app = googleplaystore.App(ps, app_id)
    reviews = []
    try:
        page = 0
        while page < num_pages:
            reviews.extend(app._get_reviews(page))
            page += 1
    except Exception:
        pass

    return reviews

In [440]:
keys_needed = ['title', 'category', 'installs', 'score', 'description']
d = {}
for appID in trending:
    old_d = api.details(appID)
    d[appID] = { key: old_d[key] for key in keys_needed }
    d[appID]['category'] = '|'.join(d[appID]['category'])
    d[appID]['reviews'] = json.dumps(get_reviews(appID, num_pages=1))

In [281]:
def get_data(collection):
    keys_needed = ['title', 'category', 'installs', 'score', 'description']
    d = {}
    for appID in collection:
        old_d = api.details(appID)
        d[appID] = {key: old_d[key] for key in keys_needed}
        d[appID]['category'] = '|'.join(d[appID]['category'])
        d[appID]['reviews'] = json.dumps(get_reviews(appID, num_pages=1))
    return d

In [400]:
top_paid[:5]

['org.twisevictory.apps',
 'com.urbandroid.lux.donate',
 'com.tmsoft.whitenoise.full',
 'com.active.aps.c25k',
 'com.abundanthealth4u.refguide4eo']

In [402]:
d

{'com.abundanthealth4u.refguide4eo': {'category': 'HEALTH_AND_FITNESS',
  'description': u'This convenient app runs on the Android family of phones and tablets.\nThis app combines information from the authoritative Reference Guide for Essential Oils, written by Connie and Alan Higley, with convenient search and note-taking features. Use the app anytime, anywhere to quickly find and confidently use recommended essential oils and oil blends for hundreds of different health and wellness conditions.\nThis app will become your go-to reference for personal essential oil use and for sharing your love of essential oils. The app features a personal guide section that details over 700 health topics for which essential oils are commonly used to help support the body\u2019s natural ability to heal itself. Additionally, key information on over 100 single oils and more than 90 commercially-available oil blends will help you better understand and use essential oils.\nThe app also includes essential o

In [441]:
df = pd.DataFrame(d).T

In [442]:
df.head()

,category,description,installs,reviews,score,title
abs.workout.fitness.tabata.hiit.stomach,HEALTH_AND_FITNESS,Abs Workout is a popular training App. We prep...,"10,000,000+","[{""rating"": ""100"", ""review-text"": "" It is eas...",4.5,"Abs Workout - Home Workout, Tabata, HIIT"
abs.workout.home.fit.belly.fat,HEALTH_AND_FITNESS,Girls want abdominal muscles(ABS) all the time...,"10,000+",[],4.4,Daily ABS Workout
alexcrusher.just6weeks.full,HEALTH_AND_FITNESS,Just 6 Weeks offers you a unique opportunity t...,"100,000+","[{""rating"": ""20"", ""review-text"": "" The app ha...",4.7,Just 6 Weeks
app.doubleagent,HEALTH_AND_FITNESS,The Best app for your workouts,100+,[],5.0,DblAge
app.quedijo,HEALTH_AND_FITNESS,¿Que dijo? permite al usuario recibir servicio...,50+,[],None,¿Que dijo?


In [443]:
df.to_csv('/Users/bettychou1993/Desktop/intern@sdl/play-scraper/categories/trending.csv', sep=',', encoding='utf-8', header=True, columns= ['title', 'category', 'installs', 'score', 'description', 'reviews'])

In [407]:
df1 = pd.read_csv('/Users/bettychou1993/Desktop/intern@sdl/play-scraper/top_paid_5_1.csv')

In [415]:
df1.iloc[0]['reviews']

'[{"rating": "100", "review-text": "  Absolutely love this app. I use it several times a day and sometimes just study the info for looking periods of time learning all I can.   Full Review   "}, {"rating": "60", "review-text": "  I love this app but it\'s not supported on my new phone the Pixel 2XL. I hope this can get fixed soon.   Full Review   "}, {"rating": "80", "review-text": "  The only reason that I don\'t love this app is that it does not save my notes that I make! Please fix this issue.   Full Review   "}, {"rating": "80", "review-text": "  I like it so far, was able to read up on additional uses for some oils. I really wish there was a way you could bookmark you\'re favorite oils for a quick reference.   Full Review   "}, {"rating": "100", "review-text": "  Absolutely thrilled now that the app has been updated to work on my new phone. This is my only go to app for oil usage and I will be recommending it to all my customers again.   Full Review   "}, {"rating": "80", "review-

In [266]:
df1['reviews'].apply(json.loads)[0][0]['review-text']

u'  Absolutely love this app. I use it several times a day and sometimes just study the info for looking periods of time learning all I can.   Full Review   '